<a href="https://colab.research.google.com/github/MariiaS/computer_vision/blob/main/First_project_Computer_Vision_Mariia_Solodiankina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Your first project submission must contain:

All the necessary instructions to execute your project.
The modifications of facedetect to detect your face from the camera on google Colab
The modifications to make faster the prediction (defining the new rectangle around the first detected face)
Your code must be:

cleaned-up of useless portions
commented (both for the existing code and the code you have added)
improved: the face detection must be done in the neighborhood of the face detect location in the previous frame and not in the full frame
Well written, modular, robust, ....

In [1]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode

import numpy as np
from PIL import Image

import io
import cv2

In [2]:
cv2.__version__ #checking the version of cv2

'4.1.2'

In [3]:
# function to stream video from the camera in the real-time

def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);
      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.20);

            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

In [5]:
def b64_to_bytes(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def bytes_to_b64(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

In [ ]:
# Load face cascades using v2.CascadeClassifier

In [6]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
!ls

--2021-02-14 21:27:29--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.06s   

2021-02-14 21:27:30 (13.8 MB/s) - ‘haarcascade_frontalface_default.xml’ saved [930127/930127]

haarcascade_frontalface_default.xml  sample_data


In [7]:
face_cascades = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

Reduce the computation time of Facedetect

In [8]:
def detect_faces(img, cascades):
  # trnasform to gray
  # use cascades to detect faces
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces = cascades.detectMultiScale(gray, 1.3, 4)
  return faces

In [9]:
# This function computes the sub region in which we would want to detect faces
# Params:
# previous: The previous face bounding box (x, y, w, h)
# image heigh and img_width: Self explaining and used to insure boundary constraints
# margin: How mach farther from the previous bounding box you would like to search for faces in the current frame 
def compute_optimized_search_region(previous, img_height, img_width, margin):
  x_new = previous[0] - margin
  y_new = previous[1] - margin
  w_new = (previous[0]+previous[2]+margin) - x_new
  h_new = (previous[1]+previous[3]+margin) - y_new
  # Don't forget to insure boundary constraints:
  x_new, y_new = max(x_new, 0), max(y_new, 0)
  w_new, h_new = min(w_new, img_width), min(h_new, img_height)
  
  return (x_new, y_new, w_new, h_new)

In [11]:
VideoCapture()
eval_js('create()')


byte = eval_js('capture()')
im = b64_to_bytes(byte)
(img_height, img_width) = im.shape[0], im.shape[1]

previous_bbox = None
margin = 50


while True:
  byte = eval_js('capture()')
  im = b64_to_bytes(byte)
  region_to_use = im

  if previous_bbox is not None: # If we have detected a face in the previous frame
    # Use the previous bounding box to compute the sub region in which you would like to seach for faces
    x_new, y_new, w_new, h_new = compute_optimized_search_region(previous_bbox, img_height, img_width, margin)
    # The new sub region
    # Draw a red rectangle around the sub region in the 'im' image (because it's the one we will plot finally)
    region_to_use = im[x_new: x_new+w_new, y_new:y_new+h_new]
    # (255,0) - red color
    cv2.rectangle(im, (x_new, y_new), (x_new+w_new, y_new+h_new), (255,0), 5)

  # detect faces in the subr egion to use
  faces = detect_faces(region_to_use, face_cascades)
  
  if len(faces) == 1:
    face = faces[0]
    # Update the face bounding box. Be careful: The coordinates should be relative the full image 'im' and not relative to the 'region_to_use'
    x,y,w,h = face if previous_bbox is None else (x_new+face[0], y_new+face[1], face[2], face[3])
    previous_bbox = (x,y,w,h)
    # (0, 255) - green
    cv2.rectangle(im, (x,y), (x+w, y+h), (0, 255), 2)
    # Draw a green rectangle around the detected face in the original image 'im'
  else: 
    previous_bbox = None # If we did not detect any face or mor ethan one face we just repeat the search using the entire image in the next frame

  eval_js('showimg("{}")'.format(bytes_to_b64(im))) # We convert our image with bounding boxes to base64 and plot it using JS 

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored